In [1]:
import torch.nn as nn
import torch

In [2]:
l = nn.Linear(16, 32)

In [3]:
c = nn.Conv2d(16, 32, kernel_size=1, padding=0, stride=1)

In [4]:
l(torch.ones(5, 16, 4,4).permute(0, 2, 3, 1))

tensor([[[[-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085]],

         [[-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085]],

         [[-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085]],

         [[-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.0554,  0.9085],
          [-0.0561,  0.9532, -1.3070,  ...,  0.8207,  0.05

In [8]:
c(torch.ones(5, 16, 4,4)).shape

torch.Size([5, 32, 4, 4])

In [10]:
c.weight.shape

torch.Size([32, 16, 1, 1])

In [11]:
l.weight.shape

torch.Size([32, 16])

In [5]:
testing_tensor = torch.ones(5, 16, 4, 4)

In [ ]:
import numpy as np
import pywt

def srf_fusion(images, level, wavelet='db2', strategy='max'):
    # images: a list of input images to be fused
    # level: the level of wavelet decomposition
    # wavelet: the type of wavelet to use
    # strategy: the fusion strategy to use (e.g. 'max' or 'mean')

    # Perform wavelet decomposition on each input image
    coeffs_list = []
    for img in images:
        coeffs = pywt.wavedec2(img, wavelet, level=level)
        coeffs_list.append(coeffs)

    # Calculate the sparse coefficients for each image
    sparse_list = []
    for coeffs in coeffs_list:
        sparse_coeffs = []
        for coeff in coeffs:
            coeff_flat = coeff.flatten()
            sparse_coeff = pywt.threshold(coeff_flat, np.std(coeff_flat) / 2)
            sparse_coeffs.append(sparse_coeff)
        sparse_list.append(sparse_coeffs)

    # Fuse the sparse coefficients using the specified strategy
    fused_sparse_coeffs = []
    for i in range(len(sparse_list[0])):
        coeff_i = []
        for j in range(len(sparse_list)):
            coeff_ij = sparse_list[j][i]
            coeff_i.append(coeff_ij)
        fused_coeff_i = np.zeros_like(coeff_i[0])
        if strategy == 'max':
            fused_coeff_i = np.max(coeff_i, axis=0)
        elif strategy == 'mean':
            fused_coeff_i = np.mean(coeff_i, axis=0)
        fused_sparse_coeffs.append(fused_coeff_i)

    # Reconstruct the fused image using the fused sparse coefficients
    fused_coeffs = []
    for coeffs in coeffs_list:
        fused_coeffs_i = []
        for i in range(len(coeffs)):
            coeff_i = coeffs[i]
            sparse_coeff_i = sparse_list[0][i]  # Use the sparse coefficients of the first image
            fused_sparse_coeff_i = fused_sparse_coeffs[i]
            fused_coeff_i = np.zeros_like(coeff_i)
            fused_coeff_i[np.abs(coeff_i) >= np.abs(sparse_coeff_i)] = coeff_i[np.abs(coeff_i) >= np.abs(sparse_coeff_i)]
            fused_coeff_i[np.abs(coeff_i) < np.abs(sparse_coeff_i)] = fused_sparse_coeff_i[np.abs(coeff_i) < np.abs(sparse_coeff_i)]
            fused_coeffs_i.append(fused_coeff_i)
        fused_coeffs.append(tuple(fused_coeffs_i))
    fused_img = pywt.waverec2(fused_coeffs, wavelet)

    return fused_img

In [ ]:
import cv2

# Load two input images
img1 = cv2.imread('image1.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('image2.jpg', cv2.IMREAD_GRAYSCALE)

# Resize the images to the same size
img1 = cv2.resize(img1, (256, 256))
img2 = cv2.resize(img2, (256, 256))

# Perform SRF image fusion
fused_img = srf_fusion([img1, img2], level=3, wavelet='db2', strategy='max')

# Display the fused image
cv2.imshow('Fused Image', fused_img

In [ ]:
import cv2

# Load two input images
img1 = cv2.imread('image1.jpg')
img2 = cv2.imread('image2.jpg')

# Resize the images to the same size
img1 = cv2.resize(img1, (256, 256))
img2 = cv2.resize(img2, (256, 256))

# Generate a weight map based on image contrast
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
blur1 = cv2.GaussianBlur(gray1, (5, 5), 0)
blur2 = cv2.GaussianBlur(gray2, (5, 5), 0)
diff = cv2.absdiff(blur1, blur2)
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
weight_map = cv2.merge([thresh, thresh, thresh])

# Perform Laplacian Pyramid Fusion
num_levels = 4
l_pyr1 = cv2.pyrDown(img1)
l_pyr2 = cv2.pyrDown(img2)
gp1 = [img1]
gp2 = [img2]
lp1 = [l_pyr1]
lp2 = [l_pyr2]
for i in range(num_levels - 1):
    l_pyr1 = cv2.pyrDown(l_pyr1)
    l_pyr2 = cv2.pyrDown(l_pyr2)
    lp1.append(l_pyr1)
    lp2.append(l_pyr2)
    g_pyr1 = cv2.subtract(gp1[i], cv2.pyrUp(l_pyr1))
    g_pyr2 = cv2.subtract(gp2[i], cv2.pyrUp(l_pyr2))
    gp1.append(g_pyr1)
    gp2.append(g_pyr2)
fused_lp = []
for i in range(num_levels):
    fused_coeff = cv2.addWeighted(lp1[i], weight_map[i], lp2[i], 1 - weight_map[i], 0)
    fused_lp.append(fused_coeff)
fused_img = fused

In [11]:
att = nn.MultiheadAttention(16, 4, batch_first=True)

In [18]:
att(testing_tensor.reshape(5, 16, -1),testing_tensor.reshape(5, 16, -1), testing_tensor.reshape(5, 16, -1))[0].shape

torch.Size([5, 16, 16])

In [23]:
testing_tensor.reshape(5, 16, None, None).shape

TypeError: reshape(): argument 'shape' must be tuple of SymInts, but found element of type NoneType at pos 3

In [38]:
torch.ones((5, 16))[:,:,None,None].repeat(1, 1, 16, 16).shape

torch.Size([5, 16, 16, 16])

In [34]:
torch.ones((5, 16))[:,:,None,None].shape

torch.Size([5, 16, 1, 1])

In [29]:
torch.ones((5, 16))[:,:,None,None].shape

torch.Size([5, 16, 1, 1])

In [48]:
l = nn.LayerNorm(16)

In [49]:
l(torch.ones((5, 16))).shape

torch.Size([5, 16])

In [ ]:
# let's implement other fusors.